## 0. Setup

In [1]:
import yaml
from pathlib import Path
from datetime import datetime

from pyspark.sql import SparkSession
from pyspark.sql import functions as F


def find_project_root():
    """Trouve la racine du projet"""
    current = Path.cwd().resolve()
    
    if current.name == "notebooks":
        candidate = current.parent
        if (candidate / "conf").exists():
            return candidate
    
    search = current
    while search != search.parent:
        if (search / "conf").exists() and (search / "conf" / "bda_project_config.yml").exists():
            return search
        search = search.parent
    
    if (current / "conf").exists():
        return current
    
    raise FileNotFoundError("Impossible de trouver la racine du projet")


PROJECT_ROOT = find_project_root()
CONFIG_PATH = PROJECT_ROOT / "conf" / "bda_project_config.yml"

with open(CONFIG_PATH, 'r', encoding='utf-8') as f:
    config = yaml.safe_load(f)

SPARK_CFG = config['spark']

print(f" Project Root: {PROJECT_ROOT}")
print(f" Config loaded: {CONFIG_PATH}")

spark = (
    SparkSession.builder
    .appName("SanityCheck-ETL-Global")
    .master(SPARK_CFG['master'])
    .config("spark.driver.memory", SPARK_CFG['driver_memory'])
    .config("spark.sql.session.timeZone", "UTC")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")

 Project Root: /home/img/BigData/Project
 Config loaded: /home/img/BigData/Project/conf/bda_project_config.yml


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/07 05:17:08 WARN Utils: Your hostname, a03-341a, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/12/07 05:17:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/12/07 05:17:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


25/12/07 05:17:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/12/07 05:17:10 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


 Spark 4.0.1 initialisé


In [2]:
MARKET_PARQUET_PATH = PROJECT_ROOT / "data" / "output" / "market_parquet"
TX_PARQUET_PATH = PROJECT_ROOT / "data" / "output" / "transactions_parquet"


print(f"Fichiers     : {MARKET_PARQUET_PATH}")
print(f" Parquet: {TX_PARQUET_PATH}")
print()
print(f"Market existe      : {'Oui' if MARKET_PARQUET_PATH.exists() else 'Non'} {MARKET_PARQUET_PATH.exists()}")
print(f"Transactions existe: {'Oui' if TX_PARQUET_PATH.exists() else 'Non'} {TX_PARQUET_PATH.exists()}")

 CHEMINS DES PARQUET
Market Parquet     : /home/img/BigData/Project/data/output/market_parquet
Transactions Parquet: /home/img/BigData/Project/data/output/transactions_parquet

Market existe      : Oui True
Transactions existe: Oui True


---

## 1. Vérification du Schéma : Prix de Marché (`market_parquet`)

On vérifie que le Parquet contient bien les colonnes attendues après l'ETL.

In [3]:
EXPECTED_MARKET_COLS = {
    "timestamp_unix",  
    "datetime",         
    "timestamp_hour",   
    "date",             
    "hour",             
    "open",             
    "high",             
    "low",              
    "close",            
    "volume"            
}

print("SCHÉMA ATTENDU : market_parquet")
print(f"Nombre de colonnes attendues : {len(EXPECTED_MARKET_COLS)}")
print(f"Colonnes : {sorted(EXPECTED_MARKET_COLS)}")

SCHÉMA ATTENDU : market_parquet
Nombre de colonnes attendues : 10
Colonnes : ['close', 'date', 'datetime', 'high', 'hour', 'low', 'open', 'timestamp_hour', 'timestamp_unix', 'volume']


In [4]:
if MARKET_PARQUET_PATH.exists():
    df_market = spark.read.parquet(str(MARKET_PARQUET_PATH))
    
    cols_market = set(df_market.columns)
    
    print("SCHÉMA RÉEL : market_parquet")
    print(f"Nombre de colonnes : {len(cols_market)}")
    print(f"Colonnes : {sorted(cols_market)}")
    print()
    
    missing_market = EXPECTED_MARKET_COLS.difference(cols_market)
    extra_market = cols_market.difference(EXPECTED_MARKET_COLS)
    
    if missing_market:
        print(f" Colonnes MANQUANTES : {missing_market}")
    if extra_market:
        print(f"  Colonnes EN PLUS (non attendues) : {extra_market}")
    
    if not missing_market and not extra_market:
        print(" Schéma market_parquet CONFORME aux attentes (colonnes identiques)")
    elif not missing_market:
        print(" Schéma market_parquet OK (toutes les colonnes attendues présentes)")
    
    print()
    print("Schema Spark :")
    df_market.printSchema()
else:
    print(" market_parquet introuvable !")
    print(" Lancez d'abord : notebooks/etl_market_price.ipynb")
    df_market = None

SCHÉMA RÉEL : market_parquet
Nombre de colonnes : 10
Colonnes : ['close', 'date', 'datetime', 'high', 'hour', 'low', 'open', 'timestamp_hour', 'timestamp_unix', 'volume']

 Schéma market_parquet CONFORME aux attentes (colonnes identiques)

Schema Spark :
root
 |-- timestamp_unix: double (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- timestamp_hour: long (nullable = true)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: double (nullable = true)



---

## 2. Vérification du Schéma : Transactions (`transactions_parquet`)

On vérifie que le Parquet contient bien les colonnes attendues après l'ETL des blocs Bitcoin.

In [5]:
# Colonnes attendues dans transactions_parquet
EXPECTED_TX_COLS = {
    "tx_id",            # Identifiant unique de la transaction
    "block_hash",       # Hash du bloc contenant la transaction
    "timestamp",        # Timestamp Unix original du bloc
    "datetime",         # Conversion en timestamp lisible
    "timestamp_hour",   # Clé de jointure (arrondi à l'heure)
    "date",             # Date extraite
    "hour",             # Heure extraite
    "n_inputs",         # Nombre d'inputs (UTXOs consommés)
    "n_outputs",        # Nombre d'outputs (destinataires)
    "amount_sats",      # Montant total en satoshis
    "block_size"        # Taille du bloc en octets
}

print("=" * 60)
print("SCHÉMA ATTENDU : transactions_parquet")
print("=" * 60)
print(f"Nombre de colonnes attendues : {len(EXPECTED_TX_COLS)}")
print(f"Colonnes : {sorted(EXPECTED_TX_COLS)}")

📊 SCHÉMA ATTENDU : transactions_parquet
Nombre de colonnes attendues : 11
Colonnes : ['amount_sats', 'block_hash', 'block_size', 'date', 'datetime', 'hour', 'n_inputs', 'n_outputs', 'timestamp', 'timestamp_hour', 'tx_id']


In [6]:
if TX_PARQUET_PATH.exists():
    df_tx = spark.read.parquet(str(TX_PARQUET_PATH))
    
    cols_tx = set(df_tx.columns)
    
    print(" SCHÉMA RÉEL : transactions_parquet")
    print(f"Nombre de colonnes : {len(cols_tx)}")
    print(f"Colonnes : {sorted(cols_tx)}")
    print()
    

    missing_tx = EXPECTED_TX_COLS.difference(cols_tx)
    extra_tx = cols_tx.difference(EXPECTED_TX_COLS)
    
    if missing_tx:
        print(f" Colonnes MANQUANTES : {missing_tx}")
    if extra_tx:
        print(f" Colonnes EN PLUS (non attendues) : {extra_tx}")
    
    if not missing_tx and not extra_tx:
        print(" Schéma transactions_parquet CONFORME aux attentes (colonnes identiques)")
    elif not missing_tx:
        print(" Schéma transactions_parquet OK (toutes les colonnes attendues présentes)")
    
    print()
    print("Schema Spark :")
    df_tx.printSchema()
else:
    print("transactions_parquet introuvable !")
    print("Lancez d'abord : notebooks/etl_bitcoin_blocks.ipynb")
    df_tx = None

 SCHÉMA RÉEL : transactions_parquet
Nombre de colonnes : 11
Colonnes : ['amount_sats', 'block_hash', 'block_size', 'date', 'datetime', 'hour', 'n_inputs', 'n_outputs', 'timestamp', 'timestamp_hour', 'tx_id']

 Schéma transactions_parquet CONFORME aux attentes (colonnes identiques)

Schema Spark :
root
 |-- tx_id: string (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- datetime: timestamp (nullable = true)
 |-- timestamp_hour: long (nullable = true)
 |-- date: date (nullable = true)
 |-- hour: integer (nullable = true)
 |-- n_inputs: integer (nullable = true)
 |-- n_outputs: integer (nullable = true)
 |-- amount_sats: long (nullable = true)
 |-- block_size: integer (nullable = true)



---

## 3. Périodes Temporelles Couvertes

On extrait la plage temporelle de chaque dataset :
- Première valeur de `datetime`
- Dernière valeur de `datetime`
- Nombre de lignes

In [7]:
market_min, market_max, market_count = None, None, 0
tx_min, tx_max, tx_count = None, None, 0

print("PÉRIODES TEMPORELLES COUVERTES")
print()

if df_market is not None:
    market_stats = df_market.agg(
        F.min("datetime").alias("min_date"),
        F.max("datetime").alias("max_date"),
        F.count("*").alias("count")
    ).collect()[0]
    
    market_min = market_stats["min_date"]
    market_max = market_stats["max_date"]
    market_count = market_stats["count"]
    
    duration_market = (market_max - market_min).days if market_max and market_min else 0
    
    print("Prix de marché (market_parquet):")
    print(f"   Première donnée : {market_min.strftime('%d/%m/%Y %H:%M:%S') if market_min else 'N/A'}")
    print(f"   Dernière donnée : {market_max.strftime('%d/%m/%Y %H:%M:%S') if market_max else 'N/A'}")
    print(f"   Durée couverte  : {duration_market:,} jours (~{duration_market // 365} ans)")
    print(f"   Nombre de lignes: {market_count:,}")
    print()
else:
    print("Prix de marché : données non disponibles")
    print()

if df_tx is not None:
    tx_stats = df_tx.agg(
        F.min("datetime").alias("min_date"),
        F.max("datetime").alias("max_date"),
        F.count("*").alias("count")
    ).collect()[0]
    
    tx_min = tx_stats["min_date"]
    tx_max = tx_stats["max_date"]
    tx_count = tx_stats["count"]
    
    duration_tx = (tx_max - tx_min).days if tx_max and tx_min else 0
    
    print(" Transactions blockchain (transactions_parquet):")
    print(f"   Première donnée : {tx_min.strftime('%d/%m/%Y %H:%M:%S') if tx_min else 'N/A'}")
    print(f"   Dernière donnée : {tx_max.strftime('%d/%m/%Y %H:%M:%S') if tx_max else 'N/A'}")
    print(f"   Durée couverte  : {duration_tx:,} jours")
    print(f"   Nombre de lignes: {tx_count:,}")
else:
    print("Transactions blockchain : données non disponibles")

📅 PÉRIODES TEMPORELLES COUVERTES



Prix de marché (market_parquet):
   Première donnée : 01/01/2012 11:01:00
   Dernière donnée : 01/12/2025 00:59:00
   Durée couverte  : 5,082 jours (~13 ans)
   Nombre de lignes: 7,317,759



 Transactions blockchain (transactions_parquet):
   Première donnée : 05/12/2013 19:19:51
   Dernière donnée : 15/01/2014 01:19:13
   Durée couverte  : 40 jours
   Nombre de lignes: 2,236,800


---

## 4. Chevauchement Temporel entre Prix et Transactions

On calcule l'**intersection temporelle** entre les deux séries :
- `début_chevauchement = max(min_prix, min_transactions)`
- `fin_chevauchement = min(max_prix, max_transactions)`

Si le chevauchement est non vide, la **jointure temporelle** (sur `timestamp_hour`) est possible.

In [8]:
print("=" * 60)
print(" CHEVAUCHEMENT TEMPOREL")
print("=" * 60)
print()

if market_min and tx_min:
    # Calcul de l'intersection
    overlap_start = max(market_min, tx_min)
    overlap_end = min(market_max, tx_max)
    
    print(" Récapitulatif des périodes :")
    print(f"   Prix         : {market_min.strftime('%d/%m/%Y')} → {market_max.strftime('%d/%m/%Y')}")
    print(f"   Transactions : {tx_min.strftime('%d/%m/%Y')} → {tx_max.strftime('%d/%m/%Y')}")
    print()
    
    if overlap_start <= overlap_end:
        # Chevauchement existe
        # +1 pour inclure le jour de début ET de fin
        overlap_days = (overlap_end.date() - overlap_start.date()).days + 1
        overlap_hours = int((overlap_end - overlap_start).total_seconds() / 3600)
        
        print(" CHEVAUCHEMENT TEMPOREL DÉTECTÉ !")
        print()
        print(f"    Période commune : {overlap_start.strftime('%d/%m/%Y %H:%M')} → {overlap_end.strftime('%d/%m/%Y %H:%M')}")
        print(f"    Durée           : {overlap_days} jour(s) ({overlap_hours:,} heures)")
        print()
        print("   ═══════════════════════════════════════════════════════")
        print("    Les deux datasets ont des données sur une période commune.")
        print("    La jointure temporelle sur 'timestamp_hour' est POSSIBLE.")
        print("   ═══════════════════════════════════════════════════════")
        print()
        print("    Prochaine étape : notebooks/feature_eng.ipynb")
        print("      → Jointure des prix et transactions par timestamp_hour")
        print("      → Création des features pour le modèle de prédiction")
    else:
        # Pas de chevauchement
        gap_days = (overlap_start - overlap_end).days
        
        print(" AUCUN CHEVAUCHEMENT TEMPOREL !")
        print()
        print(f"    Les deux datasets ne se chevauchent pas.")
        print(f"    Écart entre les deux : {gap_days} jour(s)")
        print()
        print("   ═══════════════════════════════════════════════════════")
        print("    La jointure temporelle ne produira AUCUN résultat.")
        print("   ═══════════════════════════════════════════════════════")
        print()
        print("    Solutions possibles :")
        print("      1. Télécharger des blocs Bitcoin couvrant la période des prix")
        print("      2. Télécharger des prix couvrant la période des blocs")
        print("      3. Utiliser un dataset dérivé (ex: BigQuery public dataset)")

elif market_min:
    print("  Impossible de calculer le chevauchement")
    print("   → transactions_parquet manquant ou vide")
    print("   → Lancez d'abord : notebooks/etl_bitcoin_blocks.ipynb")

else:
    print(" Impossible de calculer le chevauchement")
    print("   → Les deux Parquet sont manquants ou vides")
    print("   → Lancez d'abord : scripts/run_etl.sh")

🔗 CHEVAUCHEMENT TEMPOREL

📊 Récapitulatif des périodes :
   Prix         : 01/01/2012 → 01/12/2025
   Transactions : 05/12/2013 → 15/01/2014

✅ CHEVAUCHEMENT TEMPOREL DÉTECTÉ !

   🔗 Période commune : 05/12/2013 19:19 → 15/01/2014 01:19
   📅 Durée           : 42 jour(s) (965 heures)

   ═══════════════════════════════════════════════════════
   ✅ Les deux datasets ont des données sur une période commune.
   ✅ La jointure temporelle sur 'timestamp_hour' est POSSIBLE.
   ═══════════════════════════════════════════════════════

   💡 Prochaine étape : notebooks/feature_eng.ipynb
      → Jointure des prix et transactions par timestamp_hour
      → Création des features pour le modèle de prédiction


---

## 5. Résumé Final

In [9]:
print("=" * 60)
print(" RÉSUMÉ DU SANITY CHECK ETL GLOBAL")
print("=" * 60)
print()

# Statut market
market_ok = df_market is not None and not EXPECTED_MARKET_COLS.difference(set(df_market.columns))
print(f"{'Validé' if market_ok else 'Invalide'} market_parquet      : ", end="")
if df_market is not None:
    print(f"{len(df_market.columns)} colonnes, {market_count:,} lignes")
else:
    print("NON DISPONIBLE")

# Statut transactions
tx_ok = df_tx is not None and not EXPECTED_TX_COLS.difference(set(df_tx.columns))
print(f"{'Validé' if tx_ok else 'Invalide'} transactions_parquet : ", end="")
if df_tx is not None:
    print(f"{len(df_tx.columns)} colonnes, {tx_count:,} lignes")
else:
    print("NON DISPONIBLE")

# Statut chevauchement
overlap_ok = False
if market_min and tx_min:
    overlap_start = max(market_min, tx_min)
    overlap_end = min(market_max, tx_max)
    overlap_ok = overlap_start <= overlap_end

print(f"{'Validé' if overlap_ok else 'Invalide'} Chevauchement temporel : ", end="")
if overlap_ok:
    overlap_days = (overlap_end.date() - overlap_start.date()).days + 1
    print(f"{overlap_days} jour(s) de données communes")
else:
    print("AUCUN")

print()
print("=" * 60)

if market_ok and tx_ok and overlap_ok:
    print(" TOUS LES CHECKS PASSENT !")
    print("   → Vous pouvez continuer avec feature_eng.ipynb")
elif market_ok and overlap_ok:
    print("  CHECKS PARTIELS")
    print("   → transactions_parquet manquant ou incomplet")
    print("   → Le feature engineering sera limité aux données prix")
else:
    print(" CHECKS ÉCHOUÉS")
    print("   → Vérifiez les ETL avant de continuer")

print("=" * 60)

📋 RÉSUMÉ DU SANITY CHECK ETL GLOBAL

✅ market_parquet      : 10 colonnes, 7,317,759 lignes
✅ transactions_parquet : 11 colonnes, 2,236,800 lignes
✅ Chevauchement temporel : 42 jour(s) de données communes

🎉 TOUS LES CHECKS PASSENT !
   → Vous pouvez continuer avec feature_eng.ipynb


In [10]:
# Nettoyage
spark.stop()
print("Spark session fermée")

✅ Spark session fermée
